# Mateusz Strojek, Bartłomiej Sadza
### IiE I rok stacjonarne

## Machine Learning ćwiczenia 6

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, export_graphviz, plot_tree
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from IPython.display import Image
import pydotplus 
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score, recall_score, precision_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

In [3]:
df_air_all = pd.read_csv("zad3_Airline.csv", delimiter=";")
data = df_air_all.copy()

data.head() 

,satisfaction,Gender,Customer.Type,Age,Type.of.Travel,Class,Flight.Distance,Seat.comfort,Departure.Arrival.time.convenient,Food.and.drink,...,Online.support,Ease.of.Online.booking,On.board.service,Leg.room.service,Baggage.handling,Checkin.service,Cleanliness,Online.boarding,Departure.Delay.in.Minutes,Arrival.Delay.in.Minutes
0,satisfied,Female,Loyal Customer,20.0,Personal Travel,Eco,3009,0,1,0,...,3,4,1,2,3,1,4,4,0,0.0
1,satisfied,Female,Loyal Customer,48.0,Personal Travel,Eco,1449,0,1,0,...,2,2,2,1,2,4,2,2,0,2.0
2,satisfied,Male,NaN,NaN,Personal Travel,Eco,2840,0,1,0,...,4,4,2,4,2,4,3,4,0,0.0
3,satisfied,Male,Loyal Customer,13.0,Personal Travel,Eco,2306,0,1,0,...,3,4,4,2,3,2,2,4,0,0.0
4,satisfied,Male,Loyal Customer,34.0,Personal Travel,Eco,1952,0,1,0,...,2,2,3,4,4,4,4,2,10,0.0


### Proces przygotowywania danych do wykonania zadania

In [4]:
data.isnull().sum()
print(data.loc[:, ["Customer.Type","Age", "Gate.location", "Arrival.Delay.in.Minutes"]])

cate_na = ['Customer.Type', 'Gate.location']

for column in cate_na: 
    data[column] = data[column].fillna(data[column].mode()[0])

data_dummies = pd.get_dummies(data, drop_first=True)

          Customer.Type   Age  Gate.location  Arrival.Delay.in.Minutes
0        Loyal Customer  20.0            3.0                       0.0
1        Loyal Customer  48.0            3.0                       2.0
2                   NaN   NaN            3.0                       0.0
3        Loyal Customer  13.0            3.0                       0.0
4        Loyal Customer  34.0            3.0                       0.0
...                 ...   ...            ...                       ...
3995  disloyal Customer  20.0            4.0                       0.0
3996  disloyal Customer  68.0            4.0                       4.0
3997                NaN  26.0            2.0                       0.0
3998  disloyal Customer   7.0            4.0                       0.0
3999  disloyal Customer  23.0            2.0                       0.0

[4000 rows x 4 columns]


In [5]:
data_dropped = data.dropna()

data_dropped.isnull().sum()
data_dropped.head()

,satisfaction,Gender,Customer.Type,Age,Type.of.Travel,Class,Flight.Distance,Seat.comfort,Departure.Arrival.time.convenient,Food.and.drink,...,Online.support,Ease.of.Online.booking,On.board.service,Leg.room.service,Baggage.handling,Checkin.service,Cleanliness,Online.boarding,Departure.Delay.in.Minutes,Arrival.Delay.in.Minutes
0,satisfied,Female,Loyal Customer,20.0,Personal Travel,Eco,3009,0,1,0,...,3,4,1,2,3,1,4,4,0,0.0
1,satisfied,Female,Loyal Customer,48.0,Personal Travel,Eco,1449,0,1,0,...,2,2,2,1,2,4,2,2,0,2.0
3,satisfied,Male,Loyal Customer,13.0,Personal Travel,Eco,2306,0,1,0,...,3,4,4,2,3,2,2,4,0,0.0
4,satisfied,Male,Loyal Customer,34.0,Personal Travel,Eco,1952,0,1,0,...,2,2,3,4,4,4,4,2,10,0.0
5,satisfied,Male,Loyal Customer,70.0,Personal Travel,Eco,1650,0,1,0,...,4,4,1,4,4,4,4,4,19,45.0


In [6]:
num_na_air = ["Age", "Arrival.Delay.in.Minutes"]

data = data_dropped.copy()

for column in num_na_air:
    data[column] = data[column].fillna(data[column].mean())

data.satisfaction = data.satisfaction.map({"satisfied" : 1, "dissatisfied" : 0})
data["Customer.Type"] = data["Customer.Type"].map({"Loyal Customer" : 1, "disloyal Customer" : 0})
data.Gender = data.Gender.map({"Male" : 1, "Female" : 0})
data["Type.of.Travel"] = data["Type.of.Travel"].map({"Business travel" : 1, "Personal Travel" : 0})
data.Class = data.Class.map({"Eco" : 0, "Eco Plus" : 1, "Business" : 2})

data

,satisfaction,Gender,Customer.Type,Age,Type.of.Travel,Class,Flight.Distance,Seat.comfort,Departure.Arrival.time.convenient,Food.and.drink,...,Online.support,Ease.of.Online.booking,On.board.service,Leg.room.service,Baggage.handling,Checkin.service,Cleanliness,Online.boarding,Departure.Delay.in.Minutes,Arrival.Delay.in.Minutes
0,1,0,1,20.0,0,0,3009,0,1,0,...,3,4,1,2,3,1,4,4,0,0.0
1,1,0,1,48.0,0,0,1449,0,1,0,...,2,2,2,1,2,4,2,2,0,2.0
3,1,1,1,13.0,0,0,2306,0,1,0,...,3,4,4,2,3,2,2,4,0,0.0
4,1,1,1,34.0,0,0,1952,0,1,0,...,2,2,3,4,4,4,4,2,10,0.0
5,1,1,1,70.0,0,0,1650,0,1,0,...,4,4,1,4,4,4,4,4,19,45.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,1,1,0,20.0,0,0,2442,4,2,4,...,3,3,1,2,3,3,4,3,0,0.0
3996,1,1,0,68.0,0,0,977,4,4,4,...,2,2,2,4,4,2,4,2,0,4.0
3997,0,0,1,26.0,0,0,2539,2,4,2,...,2,2,3,5,5,3,5,3,15,0.0
3998,0,0,0,7.0,0,0,1568,3,4,0,...,4,4,4,5,3,1,4,4,0,0.0


## 1. Wykorzystać algorytm SVM do predykcji wartości zmiennej wynikowej.

### a) Zbudować model wykorzystujący jądro liniowe (linear). Przedstawić hiperparametry, które można optymalizować przy jądrze liniowym (wyjaśnić za co odpowiadają). Zweryfikować jak zmiana tych hiperparametrów wpływa na uzyskiwane wyniki.

##### Parametr C

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, recall_score, precision_score
import pandas as pd

# Przygotowanie danych
X = data.drop('satisfaction', axis=1)
y = data['satisfaction']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

# Definicja parametrów do przeszukania
param_grid = {'C': [0.01, 0.1, 1, 2, 5, 10, 100], 'kernel': ['linear']}

# Inicjalizacja GridSearchCV
grid_search = GridSearchCV(SVC(), param_grid, cv=5, scoring=['accuracy', 'recall', 'precision'], refit='accuracy', n_jobs=-1)

# Dopasowanie modelu
grid_search.fit(X_train, y_train)

# Najlepszy model
best_model = grid_search.best_estimator_

# Predykcje i wyniki
y_pred_train = best_model.predict(X_train)
y_pred_test = best_model.predict(X_test)

wyniki = {
    'C': best_model.C,
    'train_accuracy': accuracy_score(y_train, y_pred_train),
    'test_accuracy': accuracy_score(y_test, y_pred_test),
    'train_recall': recall_score(y_train, y_pred_train),
    'test_recall': recall_score(y_test, y_pred_test),
    'train_precision': precision_score(y_train, y_pred_train),
    'test_precision': precision_score(y_test, y_pred_test)
}

wyniki_df = pd.DataFrame([wyniki])
print(wyniki_df)

   C  train_accuracy  test_accuracy  train_recall  test_recall  \
0  2        0.838688        0.82027      0.819986     0.796748   

   train_precision  test_precision  
0         0.848442        0.835227  


### b) Zbudować model wykorzystujący jądro wielomianowe (poly). Przedstawić hiperparametry, które można optymalizować przy tym jądrze (wyjaśnić za co odpowiadają). Zweryfikować jak zmiana tych hiperparametrów wpływa na uzyskiwane wyniki.

In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, recall_score, precision_score

# Zakładam, że dane są już wczytane do zmiennej 'data'
X = data.drop('satisfaction', axis=1)
y = data['satisfaction']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=10)

# Definiowanie hiperparametrów do optymalizacji
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],
    'degree': [2, 3, 4, 5],
    'coef0': [0, 1, 10]
}

# Użycie GridSearchCV do znalezienia najlepszych hiperparametrów
grid_search = GridSearchCV(SVC(kernel='poly'), param_grid, cv=5, n_jobs=-1)
grid_search.fit(X_train, y_train)

print("Najlepsze parametry: ", grid_search.best_params_)
print("Najlepszy wynik: ", grid_search.best_score_)

# Ocena modelu z najlepszymi hiperparametrami
best_model = grid_search.best_estimator_
y_pred_train = best_model.predict(X_train)
y_pred_test = best_model.predict(X_test)

wyniki = {
    'train_accuracy': accuracy_score(y_train, y_pred_train),
    'test_accuracy': accuracy_score(y_test, y_pred_test),
    'train_recall': recall_score(y_train, y_pred_train), 
    'test_recall': recall_score(y_test, y_pred_test), 
    'train_precision': precision_score(y_train, y_pred_train), 
    'test_precision': precision_score(y_test, y_pred_test), 
}

wyniki_df = pd.DataFrame([wyniki])
print(wyniki_df)

Najlepsze parametry:  {'C': 10, 'coef0': 10, 'degree': 5}
Najlepszy wynik:  0.8364980017611596
   train_accuracy  test_accuracy  train_recall  test_recall  train_precision  \
0        0.842501       0.816216      0.824102     0.828877         0.852108   

   test_precision  
0        0.811518  


### c) Zbudować model wykorzystujący jądro radialne (rbf - radial basis function). Przedstawić hiperparametry, które można optymalizować przy tym jądrze (wyjaśnić za co odpowiadają). Zweryfikować jak zmiana tych hiperparametrów wpływa na uzyskiwane wyniki.

In [ ]:
# Zakładam, że dane są już wczytane do zmiennej 'data'
X = data.drop('satisfaction', axis=1)
y = data['satisfaction']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

# Definiowanie hiperparametrów do optymalizacji
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],
    'gamma': [0.001, 0.01, 0.1, 1, 10]
}

# Użycie GridSearchCV do znalezienia najlepszych hiperparametrów
grid_search = GridSearchCV(SVC(kernel='rbf'), param_grid, cv=5, n_jobs=-1)
grid_search.fit(X_train, y_train)

print("Najlepsze parametry: ", grid_search.best_params_)
print("Najlepszy wynik: ", grid_search.best_score_)

# Ocena modelu z najlepszymi hiperparametrami
best_model = grid_search.best_estimator_
y_pred_train = best_model.predict(X_train)
y_pred_test = best_model.predict(X_test)

wyniki = {
    'train_accuracy': accuracy_score(y_train, y_pred_train),
    'test_accuracy': accuracy_score(y_test, y_pred_test),
    'train_recall': recall_score(y_train, y_pred_train, pos_label='satisfied'),
    'test_recall': recall_score(y_test, y_pred_test, pos_label='satisfied'),
    'train_precision': precision_score(y_train, y_pred_train, pos_label='satisfied'),
    'test_precision': precision_score(y_test, y_pred_test, pos_label='satisfied')
}

wyniki_df = pd.DataFrame([wyniki])
print(wyniki_df)